In [1]:
import logging
import colorlog
import random
import itertools

from ipywidgets import IntProgress
from IPython.display import display

import os
import django

In [2]:
random.seed(56)

In [3]:
# Setup logging 
logger = logging.getLogger()
handler = logging.StreamHandler()
handler.setFormatter(
    colorlog.ColoredFormatter(
        (
            '%(asctime)s.%(msecs)03d '                  # time
            '%(log_color)s%(levelname)-7s%(reset)s '    # level
            # '%(log_color)s[%(funcName)s]%(reset)s '   # where log is generated
            '%(message)s'                               # message
        ),
        '%Y-%m-%d %H:%M:%S',
        log_colors=colorlog.default_log_colors
    )
)
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logging.captureWarnings(True)

In [4]:
os.environ.setdefault('DJANGO_SETTINGS_MODULE', "logo_proposal.settings.local")

'logo_proposal.settings.local'

In [5]:
django.setup()

In [6]:
from proposals.models import LogoProposal
from reviews.models import Review
from users.models import EmailUser

In [7]:
reviewers = (
    EmailUser.objects.all()
    .exclude(email__in=['lding@wustl.edu', 'webmaster@liang2.io'])
)

In [8]:
len(list(reviewers))

2016-09-27 16:15:49.549 DEBUG   (0.001) SELECT "users_emailuser"."id", "users_emailuser"."password", "users_emailuser"."last_login", "users_emailuser"."is_superuser", "users_emailuser"."email", "users_emailuser"."name", "users_emailuser"."verified", "users_emailuser"."is_staff", "users_emailuser"."is_active", "users_emailuser"."date_joined" FROM "users_emailuser" WHERE NOT ("users_emailuser"."email" IN ('lding@wustl.edu', 'webmaster@liang2.io')); args=('lding@wustl.edu', 'webmaster@liang2.io')


22

In [9]:
from django.conf import settings

In [10]:
def assign_review(reviewer):
    if len(reviewer.reviews.all()) >= settings.NUM_REVIEWS_PER_USER:
        raise ValueError(
            "Reviewer %s have exceeded the maximum number of reviews." % reviewer
        )
    reviewed_proposals = [review.proposal_id for review in reviewer.reviews.all()]
    unseen_proposals = (
        LogoProposal.objects
        .exclude(id__in=reviewed_proposals)
        .exclude(owner_id=reviewer)
        .prefetch_related('reviews')
    )
    unseen_proposals_count = [
        p.reviews.count() for p in unseen_proposals
    ]
    weighted_unseen_proposals = list(itertools.chain.from_iterable([
        itertools.repeat(p, 3 - count)
        for p, count in zip(unseen_proposals, unseen_proposals_count)
        if count < 3
    ]))
    if not unseen_proposals:
        messages.warning(
            request,
            'All available proposals have been reviewed.'
        )
        return None
    proposal_to_review = None
    if weighted_unseen_proposals:
        # means there are proposals to draw
        proposal_to_review = random.choice(weighted_unseen_proposals)
    else:
        # all proposals are sufficiently reviewed
        logger.info('All proposasls are sufficiently reviewed')
        
        proposal_to_review = random.choice(unseen_proposals)

    review = Review.objects.create(
        reviewer=reviewer,
        proposal=proposal_to_review,
    )
    return review

In [11]:
# First assignment
for reviewer in reviewers:
    assign_review(reviewer)

2016-09-27 16:15:49.606 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."reviewer_id" = 2; args=(2,)
2016-09-27 16:15:49.609 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."reviewer_id" = 2; args=(2,)
2016-09-27 16:15:49.611 DEBUG   (0.000) SELECT "proposals_logoprop

In [12]:
# 2nd assignment
for reviewer in reviewers:
    assign_review(reviewer)

2016-09-27 16:15:50.203 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."reviewer_id" = 2; args=(2,)
2016-09-27 16:15:50.206 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."reviewer_id" = 2; args=(2,)
2016-09-27 16:15:50.208 DEBUG   (0.000) SELECT "proposals_logoprop

In [13]:
# 3rd assignment
for reviewer in reviewers:
    assign_review(reviewer)

2016-09-27 16:15:50.659 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."reviewer_id" = 2; args=(2,)
2016-09-27 16:15:50.663 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."reviewer_id" = 2; args=(2,)
2016-09-27 16:15:50.666 DEBUG   (0.001) SELECT "proposals_logoprop

In [14]:
# check the review status
for reviewer in reviewers:
    to_review = ', '.join(
        '#%d (by %d)' % (p.proposal.pk, p.proposal.owner.pk) for p in reviewer.reviews.all()
    )
    
    print('Reviewer #{:d} is reviewing {}'.format(reviewer.pk, to_review))

2016-09-27 16:15:51.136 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."reviewer_id" = 2; args=(2,)
2016-09-27 16:15:51.139 DEBUG   (0.000) SELECT "proposals_logoproposal"."id", "proposals_logoproposal"."owner_id", "proposals_logoproposal"."logo", "proposals_logoproposal"."description" FROM "proposals_logoproposal" WHERE "proposals_logoproposal"."id" = 28; args=(28,)
2016-09-27 16:15:51.141 DEBUG   (0.000) SELECT "users_emailuser"."id", "users_emailuser"."password", "users_emailuser"."last_login", "users_emailuser"."is_superuser", "users_emailuser"."email", "users_emailuser"."name", "users_emailuser"."verified", "users_emailu

Reviewer #2 is reviewing #28 (by 21), #21 (by 17), #12 (by 14)
Reviewer #3 is reviewing #2 (by 2), #19 (by 16), #9 (by 10)
Reviewer #4 is reviewing #23 (by 18), #6 (by 7), #13 (by 14)
Reviewer #5 is reviewing #26 (by 3), #14 (by 8), #6 (by 7)
Reviewer #6 is reviewing #17 (by 15), #26 (by 3), #22 (by 17)
Reviewer #7 is reviewing #28 (by 21), #3 (by 2), #18 (by 16)
Reviewer #8 is reviewing #22 (by 17), #7 (by 9), #25 (by 19)
Reviewer #9 is reviewing #14 (by 8), #16 (by 15), #24 (by 19)
Reviewer #10 is reviewing #6 (by 7), #12 (by 14), #16 (by 15)
Reviewer #11 is reviewing #21 (by 17), #9 (by 10), #2 (by 2)
Reviewer #12 is reviewing #5 (by 6), #7 (by 9), #27 (by 3)
Reviewer #13 is reviewing #20 (by 16), #27 (by 3), #28 (by 21)
Reviewer #14 is reviewing #20 (by 16), #10 (by 11), #4 (by 5)
Reviewer #15 is reviewing #8 (by 9), #25 (by 19), #26 (by 3)
Reviewer #16 is reviewing #16 (by 15), #11 (by 13), #21 (by 17)
Reviewer #17 is reviewing #7 (by 9), #20 (by 16), #3 (by 2)


2016-09-27 16:15:51.355 DEBUG   (0.000) SELECT "proposals_logoproposal"."id", "proposals_logoproposal"."owner_id", "proposals_logoproposal"."logo", "proposals_logoproposal"."description" FROM "proposals_logoproposal" WHERE "proposals_logoproposal"."id" = 2; args=(2,)
2016-09-27 16:15:51.356 DEBUG   (0.000) SELECT "users_emailuser"."id", "users_emailuser"."password", "users_emailuser"."last_login", "users_emailuser"."is_superuser", "users_emailuser"."email", "users_emailuser"."name", "users_emailuser"."verified", "users_emailuser"."is_staff", "users_emailuser"."is_active", "users_emailuser"."date_joined" FROM "users_emailuser" WHERE "users_emailuser"."id" = 2; args=(2,)
2016-09-27 16:15:51.359 DEBUG   (0.000) SELECT "proposals_logoproposal"."id", "proposals_logoproposal"."owner_id", "proposals_logoproposal"."logo", "proposals_logoproposal"."description" FROM "proposals_logoproposal" WHERE "proposals_logoproposal"."id" = 10; args=(10,)
2016-09-27 16:15:51.360 DEBUG   (0.000) SELECT "user

Reviewer #18 is reviewing #2 (by 2), #10 (by 11), #29 (by 20)
Reviewer #19 is reviewing #23 (by 18), #18 (by 16), #4 (by 5)
Reviewer #20 is reviewing #18 (by 16), #8 (by 9), #24 (by 19)
Reviewer #21 is reviewing #8 (by 9), #4 (by 5), #29 (by 20)
Reviewer #23 is reviewing #3 (by 2), #13 (by 14), #10 (by 11)
Reviewer #24 is reviewing #29 (by 20), #27 (by 3), #25 (by 19)


In [15]:
for proposal in LogoProposal.objects.all():
    print(
        'Proposal #{:d} reviewed by {}'
        .format(proposal.pk, ', '.join(str(r.reviewer.pk) for r in proposal.reviews.all()))
    )    

2016-09-27 16:15:51.437 DEBUG   (0.000) SELECT "proposals_logoproposal"."id", "proposals_logoproposal"."owner_id", "proposals_logoproposal"."logo", "proposals_logoproposal"."description" FROM "proposals_logoproposal"; args=()
2016-09-27 16:15:51.440 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."proposal_id" = 2; args=(2,)
2016-09-27 16:15:51.442 DEBUG   (0.000) SELECT "users_emailuser"."id", "users_emailuser"."password", "users_emailuser"."last_login", "users_emailuser"."is_superuser", "users_emailuser"."email", "users_emailuser"."name", "users_emailuser"."verified", "users_emailuser"."is_staff", "users_emailuser"."is_activ

Proposal #2 reviewed by 3, 18, 11
Proposal #3 reviewed by 23, 7, 17
Proposal #4 reviewed by 21, 14, 19
Proposal #5 reviewed by 12
Proposal #6 reviewed by 10, 4, 5
Proposal #7 reviewed by 17, 8, 12
Proposal #8 reviewed by 15, 21, 20
Proposal #9 reviewed by 11, 3
Proposal #10 reviewed by 14, 18, 23
Proposal #11 reviewed by 16
Proposal #12 reviewed by 10, 2
Proposal #13 reviewed by 23, 4
Proposal #14 reviewed by 9, 5
Proposal #16 reviewed by 16, 9, 10
Proposal #17 reviewed by 6
Proposal #18 reviewed by 20, 19, 7
Proposal #19 reviewed by 3
Proposal #20 reviewed by 13, 14, 17
Proposal #21 reviewed by 11, 2, 16


2016-09-27 16:15:51.567 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."proposal_id" = 22; args=(22,)
2016-09-27 16:15:51.569 DEBUG   (0.000) SELECT "users_emailuser"."id", "users_emailuser"."password", "users_emailuser"."last_login", "users_emailuser"."is_superuser", "users_emailuser"."email", "users_emailuser"."name", "users_emailuser"."verified", "users_emailuser"."is_staff", "users_emailuser"."is_active", "users_emailuser"."date_joined" FROM "users_emailuser" WHERE "users_emailuser"."id" = 8; args=(8,)
2016-09-27 16:15:51.571 DEBUG   (0.000) SELECT "users_emailuser"."id", "users_emailuser"."password", "users_emailuser"."l

Proposal #22 reviewed by 8, 6
Proposal #23 reviewed by 4, 19
Proposal #24 reviewed by 9, 20
Proposal #25 reviewed by 15, 8, 24
Proposal #26 reviewed by 5, 6, 15
Proposal #27 reviewed by 13, 24, 12
Proposal #28 reviewed by 2, 7, 13
Proposal #29 reviewed by 24, 18, 21


In [16]:
# 4th assignment
for reviewer in reviewers:
    assign_review(reviewer)

2016-09-27 16:15:51.634 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."reviewer_id" = 2; args=(2,)
2016-09-27 16:15:51.636 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."reviewer_id" = 2; args=(2,)
2016-09-27 16:15:51.639 DEBUG   (0.000) SELECT "proposals_logoprop

In [17]:
for proposal in LogoProposal.objects.all():
    print(
        'Proposal #{:d} reviewed by {}'
        .format(proposal.pk, ', '.join(str(r.reviewer.pk) for r in proposal.reviews.all()))
    )    

2016-09-27 16:16:27.892 DEBUG   (0.000) SELECT "proposals_logoproposal"."id", "proposals_logoproposal"."owner_id", "proposals_logoproposal"."logo", "proposals_logoproposal"."description" FROM "proposals_logoproposal"; args=()
2016-09-27 16:16:27.895 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."proposal_id" = 2; args=(2,)
2016-09-27 16:16:27.899 DEBUG   (0.000) SELECT "users_emailuser"."id", "users_emailuser"."password", "users_emailuser"."last_login", "users_emailuser"."is_superuser", "users_emailuser"."email", "users_emailuser"."name", "users_emailuser"."verified", "users_emailuser"."is_staff", "users_emailuser"."is_activ

Proposal #2 reviewed by 3, 18, 11, 21
Proposal #3 reviewed by 23, 7, 17
Proposal #4 reviewed by 21, 14, 19
Proposal #5 reviewed by 12, 13, 15
Proposal #6 reviewed by 10, 4, 5, 20
Proposal #7 reviewed by 17, 8, 12
Proposal #8 reviewed by 15, 21, 20
Proposal #9 reviewed by 11, 3, 16, 23
Proposal #10 reviewed by 14, 18, 23
Proposal #11 reviewed by 16, 8, 14, 24
Proposal #12 reviewed by 10, 2, 4
Proposal #13 reviewed by 23, 4, 7
Proposal #14 reviewed by 9, 5, 2, 17
Proposal #16 reviewed by 16, 9, 10
Proposal #17 reviewed by 6, 5, 11
Proposal #18 reviewed by 20, 19, 7
Proposal #19 reviewed by 3, 10, 12, 19
Proposal #20 reviewed by 13, 14, 17
Proposal #21 reviewed by 11, 2, 16
Proposal #22 reviewed by 8, 6, 9
Proposal #23 reviewed by 4, 19, 6


2016-09-27 16:16:28.110 DEBUG   (0.001) SELECT "users_emailuser"."id", "users_emailuser"."password", "users_emailuser"."last_login", "users_emailuser"."is_superuser", "users_emailuser"."email", "users_emailuser"."name", "users_emailuser"."verified", "users_emailuser"."is_staff", "users_emailuser"."is_active", "users_emailuser"."date_joined" FROM "users_emailuser" WHERE "users_emailuser"."id" = 3; args=(3,)
2016-09-27 16:16:28.114 DEBUG   (0.000) SELECT "reviews_review"."id", "reviews_review"."reviewer_id", "reviews_review"."proposal_id", "reviews_review"."significance", "reviews_review"."innovation", "reviews_review"."approach", "reviews_review"."investigator", "reviews_review"."environment", "reviews_review"."overall_impact", "reviews_review"."comment", "reviews_review"."comment_disclosure" FROM "reviews_review" WHERE "reviews_review"."proposal_id" = 25; args=(25,)
2016-09-27 16:16:28.116 DEBUG   (0.000) SELECT "users_emailuser"."id", "users_emailuser"."password", "users_emailuser"."l

Proposal #24 reviewed by 9, 20, 3
Proposal #25 reviewed by 15, 8, 24
Proposal #26 reviewed by 5, 6, 15
Proposal #27 reviewed by 13, 24, 12
Proposal #28 reviewed by 2, 7, 13, 18
Proposal #29 reviewed by 24, 18, 21
